# Association Rule Mining - Market Basket Analysis with Mlxtend

# 1. Intro

**Have you ever considered the techniques behind the online recommendation systems or product placement strategies ?** <br><br>
**If your answer is 'Yes', you are definetely at right place !**

<img width="1000" height="400" alt="netflix" align="left" src="https://user-images.githubusercontent.com/36535914/78720348-369df700-792e-11ea-9950-8f5731409171.png">
***

Association rule learning is a rule-based machine learning method for discovering relations between variables in large databases. Goal is to identify strong relations discovered in datasets using some measures such as confidence or lift.

An association rule is an implication expression of the form X→Y, where X and Y are seperate itemsets. A more concrete example based on consumer behaviour would be {Diapers}→{Beer} suggesting that people who buy diapers are also likely to buy beer. To evaluate the "interest" of such an association rule, different metrics have been developed. The current implementation make use of the confidence and lift metrics as which we just mentioned above

- ***If a customer buys bread, he’s 70% likely of buying milk***

In the above association rule, bread is the antecedent and milk is the consequent. Simply put, it can be understood as a retail store’s association rule to target their customers better. If the above rule is a result of a thorough analysis of some data sets, it can be used to not only improve customer service but also improve the company’s revenue.

Additional to above the association rule based learning techniques are also used in many applications such as recommendation systems, medical diagnosis, protein sequence, census data or even crime prevention, [click here for details ](https://www.upgrad.com/blog/most-common-examples-of-data-mining/)

The Apriori algorithm is one of the main technique of association rule mining, which can be basically descibed as finding the most frequent itemsets in a dataset. We will be using that algorithm as well in our tutorial.

So, keep reading for details ! 

# 2. Data Load

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import networkx as nx
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori,association_rules
import matplotlib.pyplot as plt
plt.style.use('default')

In [ ]:
data = pd.read_csv("../input/d/hemanthkumar05/market-basket-optimization/Market_Basket_Optimisation.csv", header=None)
data.shape

In [ ]:
data.head()

In [ ]:
data.describe()

In [ ]:
data[1]

# 3. Data Visualizations

***- The most demanded items in dataset / Top10***

In [ ]:
# 1. Gather All Items of Each Transactions into Numpy Array
transaction = []
for i in range(0, data.shape[0]):
    for j in range(0, data.shape[1]):
        transaction.append(data.values[i,j])

transaction = np.array(transaction)

# 2. Transform Them a Pandas DataFrame
df = pd.DataFrame(transaction, columns=["items"]) 
df["incident_count"] = 1 # Put 1 to Each Item For Making Countable Table, to be able to perform Group By

# 3. Delete NaN Items from Dataset
indexNames = df[df['items'] == "nan" ].index
df.drop(indexNames , inplace=True)

# 4. Final Step: Make a New Appropriate Pandas DataFrame for Visualizations  
df_table = df.groupby("items").sum().sort_values("incident_count", ascending=False).reset_index()

# 5. Initial Visualizations
df_table.head(10).style.background_gradient(cmap='Blues')

***- The most demanded items in dataset / Top30***

In [ ]:
df_table["all"] = "all" # to have a same origin

fig = px.treemap(df_table.head(30), path=['all', "items"], values='incident_count',
                  color=df_table["incident_count"].head(30), hover_data=['items'],
                  color_continuous_scale='Blues',
                  )
fig.show()

***- Lets check whether the items have multiple records in a transaction or not***<br>
***- If the answer is "Yes", we need to handle them since they might mislead the apriori algorithm in further steps***

In [ ]:
# Transform Every Transaction to Seperate List & Gather Them into Numpy Array
# By Doing So, We Will Be Able To Iterate Through Array of Transactions

transaction = []
for i in range(data.shape[0]):
    transaction.append([str(data.values[i,j]) for j in range(data.shape[1])])
    
transaction = np.array(transaction)

# Create a DataFrame In Order To Check Status of Top20 Items

top20 = df_table["items"].head(20).values
array = []
df_top20_multiple_record_check = pd.DataFrame(columns=top20)

for i in range(0, len(top20)):
    array = []
    for j in range(0,transaction.shape[0]):
        array.append(np.count_nonzero(transaction[j]==top20[i]))
        if len(array) == len(data):
            df_top20_multiple_record_check[top20[i]] = array
        else:
            continue
            

df_top20_multiple_record_check.head(10)


In [ ]:
df_top20_multiple_record_check.describe()

- ***As you see above, only Chocolate has a max value of 2. The others have max value of 1. From that reason we can say that the data is homogenous, we can proceed without any inference.***

***- Choice Analysis / Customers' First Choices***

In [ ]:
# 1. Gather Only First Choice of Each Transactions into Numpy Array
# Similar Pattern to Above, Only Change is the Column Number "0" in Append Function
transaction = []
for i in range(0, data.shape[0]):
    transaction.append(data.values[i,0])

transaction = np.array(transaction)

# 2. Transform Them a Pandas DataFrame
df_first = pd.DataFrame(transaction, columns=["items"])
df_first["incident_count"] = 1

# 3. Delete NaN Items from Dataset
indexNames = df_first[df_first['items'] == "nan" ].index
df_first.drop(indexNames , inplace=True)

# 4. Final Step: Make a New Appropriate Pandas DataFrame for Visualizations  
df_table_first = df_first.groupby("items").sum().sort_values("incident_count", ascending=False).reset_index()
df_table_first["food"] = "food"
df_table_first = df_table_first.truncate(before=-1, after=15) # Fist 15 Choice


In [ ]:
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['figure.figsize'] = (20, 20)
first_choice = nx.from_pandas_edgelist(df_table_first, source = 'food', target = "items", edge_attr = True)
pos = nx.spring_layout(first_choice)
nx.draw_networkx_nodes(first_choice, pos, node_size = 12500, node_color = "lavender")
nx.draw_networkx_edges(first_choice, pos, width = 3, alpha = 0.6, edge_color = 'black')
nx.draw_networkx_labels(first_choice, pos, font_size = 18, font_family = 'sans-serif')
plt.axis('off')
plt.grid()
plt.title('Top First Choices', fontsize = 25)
plt.show()

***- Choice Analysis / Customers' Second Choices***

In [ ]:
# 1. Gather Only Second Choice of Each Transaction into Numpy Array

transaction = []
for i in range(0, data.shape[0]):
    transaction.append(data.values[i,1])

transaction = np.array(transaction)

# 2. Transform Them a Pandas DataFrame
df_second = pd.DataFrame(transaction, columns=["items"]) 
df_second["incident_count"] = 1

# 3. Delete NaN Items from Dataset
indexNames = df_second[df_second['items'] == "nan" ].index
df_second.drop(indexNames , inplace=True)

# 4. Final Step: Make a New Appropriate Pandas DataFrame for Visualizations  
df_table_second = df_second.groupby("items").sum().sort_values("incident_count", ascending=False).reset_index()
df_table_second["food"] = "food"
df_table_second = df_table_second.truncate(before=-1, after=15) # Fist 15 Choice


In [ ]:
import warnings
warnings.filterwarnings('ignore')

second_choice = nx.from_pandas_edgelist(df_table_second, source = 'food', target = "items", edge_attr = True)
pos = nx.spring_layout(second_choice)
nx.draw_networkx_nodes(second_choice, pos, node_size = 12500, node_color = "honeydew")
nx.draw_networkx_edges(second_choice, pos, width = 3, alpha = 0.6, edge_color = 'black')
nx.draw_networkx_labels(second_choice, pos, font_size = 18, font_family = 'sans-serif')
plt.rcParams['figure.figsize'] = (20, 20)
plt.axis('off')
plt.grid()
plt.title('Top Second Choices', fontsize = 25)
plt.show()

***- Choice Analysis / Customers' Third Choices***

In [ ]:
# 1. Gather Only Third Choice of Each Transaction into Numpy Array
## For Column "2"
transaction = []
for i in range(0, data.shape[0]):
    transaction.append(data.values[i,2])

transaction = np.array(transaction)

# 2. Transform Them a Pandas DataFrame
df_third = pd.DataFrame(transaction, columns=["items"]) # Transaction Item Name
df_third["incident_count"] = 1 # Put 1 to Each Item For Making Countable Table, Group By Will Be Done Later On

# 3. Delete NaN Items from Dataset
indexNames = df_third[df_third['items'] == "nan" ].index
df_third.drop(indexNames , inplace=True)

# 4. Final Step: Make a New Appropriate Pandas DataFrame for Visualizations  
df_table_third = df_third.groupby("items").sum().sort_values("incident_count", ascending=False).reset_index()
df_table_third["food"] = "food"
df_table_third = df_table_third.truncate(before=-1, after=15) # Fist 15 Choice


In [ ]:
fig = go.Figure(data=[go.Bar(x=df_table_third["items"], y=df_table_third["incident_count"],
            hovertext=df_table_third["items"], text=df_table_third["incident_count"], textposition="outside")])

fig.update_traces(marker_color='rgb(158,202,225)', marker_line_color='rgb(8,48,107)',
                  marker_line_width=1.5, opacity=0.65)
fig.update_layout(title_text="Customers' Third Choices", template="plotly_dark")
fig.show()

# 4. Data Pre-Processing

***In order to be able to use apriori algorithm and get most frequent itemsets, we have to transform our dataset into a 1 – 0 matrix where rows are transactions and columns are products. In that matrix, “1” should be encoded if the product has been bought on that transaction and “0” should be encoded if the product has not been bought on that transaction. This preprocessing is required for use of the algorithm.***


http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/ <br>
http://rasbt.github.io/mlxtend/user_guide/preprocessing/TransactionEncoder/


In [ ]:
# Transform Every Transaction to Seperate List & Gather Them into Numpy Array

transaction = []
for i in range(data.shape[0]):
    transaction.append([str(data.values[i,j]) for j in range(data.shape[1])])
    
transaction = np.array(transaction)
transaction

In [ ]:
te = TransactionEncoder()
te_ary = te.fit(transaction).transform(transaction)
dataset = pd.DataFrame(te_ary, columns=te.columns_)
dataset

In [ ]:
dataset.shape

***We have 121 columns/features at the moment. Extracting the most frequent itemsets from 121 feature would be compelling for a start*** <br>
***From that reason, we will start with Top 50 items which are already illustrated in Section-3***



In [ ]:
first50 = df_table["items"].head(50).values # Select Top50
dataset = dataset.loc[:,first50] # Extract Top50
dataset

In [ ]:
dataset.columns
# We extracted first 50 column successfully.

In [ ]:
# Convert dataset into 1-0 encoding

def encode_units(x):
    if x == False:
        return 0 
    if x == True:
        return 1
    
dataset = dataset.applymap(encode_units)
dataset.head(10)

***Data preprocessign is completed.***


# 5. Algorithm Implementation

## 5.1. Main Concepts of Association Rules / Apriori Algorithm

### 5.1.2. Support

*Support is an indication of how frequently the itemset appears in the dataset.*<br>
*In other words, this is an indication of how popular an itemset is in a dataset*
   
![Support](https://user-images.githubusercontent.com/36535914/81746687-98163000-94af-11ea-834a-8d9577f28930.png)

### 5.1.2. Confidence

*Confidence is an indication of how often the rule has been found to be true* <br>
*In other words, confidence says how likely item Y is purchased when item X is purchased*

![Confidence](https://user-images.githubusercontent.com/36535914/82349639-094f6900-9a03-11ea-8163-8f4d9de06e14.png)

### 5.1.3. Lift

*Lift is a metric to measure the ratio of X and Y occur together to X and Y occurrence if they were statistically independent. In other words, lift illustrates how likely item Y is purchased when item X is purchased, while controlling for how popular item Y is.*

* A Lift score that is close to 1 indicates that the antecedent and the consequent are independent and occurrence of antecedent has no impact on occurrence of consequent.

* A Lift score that is bigger than 1 indicates that the antecedent and consequent are dependent to each other, and the occurrence of antecedent has a positive impact on occurrence of consequent.

* A Lift score that is smaller than 1 indicates that the antecedent and the consequent are substitute each other that means the existence of antecedent has a negative impact to consequent or visa versa.

***Therefore having lift bigger than 1 is critial for proving associations***

![Lift](https://user-images.githubusercontent.com/36535914/85957011-5de6ec00-b992-11ea-8c5e-243f449d0241.png)


### 5.1.4. Conviction

*Conviction measures the implication strength of the rule from statistical independence
Conviction score is a ratio between the probability that X occurs without Y while they were dependent and the actual probability of X existence without Y. For instance; if (French fries) --> (beer) association has a conviction score of 1.8; the rule would be incorrect 1.8 times as often (80% more often) if the association between totally independent.*



![Conviction](https://user-images.githubusercontent.com/36535914/82484880-42f7a100-9ae3-11ea-8219-cd27a9d96d6f.png)

### 5.1.5. Consequents & Antecedents

*The IF component of an association rule is known as the antecedent. The THEN component is known as the consequent. The antecedent and the consequent are disjoint; they have no items in common.*

![Cons_Ant](https://user-images.githubusercontent.com/36535914/82491863-efd71b80-9aed-11ea-80d5-2ffebd16251d.png)

## 5.2. Implementation

***The most widely used library for Association Rules Learning implementations is 'Mlxtend'. We will be using that library as well***

In [ ]:
# Extracting the most frequest itemsets via Mlxtend.
# The length column has been added to increase ease of filtering.

frequent_itemsets = apriori(dataset, min_support=0.01, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets

***We can easily explore the itemsets via below snippes***

In [ ]:
frequent_itemsets[ (frequent_itemsets['length'] == 2) &
                   (frequent_itemsets['support'] >= 0.05) ]

In [ ]:
frequent_itemsets[ (frequent_itemsets['length'] == 3) ].head()

***Now, we will use the extracted frequent itemsets in rule creation***

In [ ]:
# We can create our rules by defining metric and its threshold.

# For a start, 
#      We set our metric as "Lift" to define whether antecedents & consequents are dependent our not.
#      Treshold is selected as "1.2" since it is required to have lift scores above than 1 if there is dependency.

rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.2)
rules["antecedents_length"] = rules["antecedents"].apply(lambda x: len(x))
rules["consequents_length"] = rules["consequents"].apply(lambda x: len(x))
rules.sort_values("lift",ascending=False)

***According to above table, we can easily say that the dependency between (herb & pepper) and (ground beef) is high since lift score is approximately 2.5x of threshold and the confidence score is promising (32%)***

***In order to get more insights from the data, let’s look into confidence!***

In [ ]:
# Sort values based on confidence

rules.sort_values("confidence",ascending=False)

- ***According to above table, the customers who bought (eggs, ground beef) is expected to buy (mineral water) with a likelihood of 50% (confidence). Lift & conviction scores support that hypothesis too*** 
- ***It would be better to keep them close to increase sales !***

***Since the most demanded product is mineral water in the dataset, the association results are mainly dominated by it. From that reason, to get more insights, it’s better to create a confidence table excluding the mineral water***

In [ ]:
rules[~rules["consequents"].str.contains("mineral water", regex=False) & 
      ~rules["antecedents"].str.contains("mineral water", regex=False)].sort_values("confidence", ascending=False).head(10)

***According to mineral water excluded table above, we can say that there is a significant relationship between ground beef and spaghetti, red wine and spaghetti. Lift and conviction scores supports that too***

***As you might have noticed, ground beef is on the top in both mineral water included and excluded table. From that reason, in order to catch new associations related to ground beef and boost the sales, let’s look into associations where ground beef is antecedent.***

In [ ]:
rules[rules["antecedents"].str.contains("ground beef", regex=False) & rules["antecedents_length"] == 1].sort_values("confidence", ascending=False).head(10)

- ***There are many associations with high confidence and lift score. We are on the right way!***

# 6. Results

***As you seen on above investigations, the flexibility of the algorithm and the mlxtend library is high therefore we can easily investigate different aspects and get new associations from the data. From that reason, the investigations could be further detailed by taking other products (rest of the Top50) into calculation or changing the criteria threshold. Nevertheless, since the association rule learning has an iterative schema, data understanding and interpretation skills and activities are really important. In that case, we should give enough importance to data visualization and/or data cleansing (if required) steps to be sure we are on the right way.***


# 7. Useful Links

- https://www.kdnuggets.com/2016/04/association-rules-apriori-algorithm-tutorial.html
- http://www.borgelt.net/doc/apriori/apriori.html#cvct
- http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/
- https://stackoverflow.com/questions/11350770/select-by-partial-string-from-a-pandas-dataframe
- https://en.wikipedia.org/wiki/Association_rule_learning
- https://www.upgrad.com/blog/association-rule-mining-an-overview-and-its-applications/
- https://www.scss.tcd.ie/Khurshid.Ahmad/Teaching/Lectures_on_Financial_Informatics/Zookeeper_Forward_Chains.pdf
- https://docs.oracle.com/cd/B28359_01/datamine.111/b28129/algo_apriori.htm#BGBCDHEB